### Generate figures for the published article

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
import matplotlib as mpl
import seaborn as sns
from shapely import wkt

#### Part 1: Descriptive Stats

In [ ]:
y_bike='km_share_cycling'
y_walk='km_share_on_foot'
model_type='beta'

# where to save figures
figpath = '../figures/'
if not os.path.exists(figpath): os.mkdir(figpath)

# where stan coefficients and other estimates are saved
outputpath = '../stan_output/' 

In [ ]:
# load in data from Stan_Model_Data_Prep
df_city = pd.read_csv('data/data_23.csv', index_col='feature_id')
df_country = pd.read_csv('data/country_23.csv', index_col='country')

# add in geometries for country-level data
countries = gpd.read_file('data/WBcountries.gpkg')
countries.loc[countries.NAME_EN=='United States of America','NAME_EN']='United States'
df_country = countries.set_index('NAME_EN')[['geometry']].join(df_country, how='left')

# add in geometries for city-level data
df_city['geometry'] = df_city.centroid.apply(wkt.loads)
df_city = gpd.GeoDataFrame(df_city, geometry='geometry')
df_city.crs = 'EPSG:4326'
df_city.to_crs(df_country.crs, inplace=True)

In [ ]:
for mode in ['cycling', 'on_foot']:
    bounds = {'cycling':[0.0,0.1], 'on_foot':[0.05,0.15]}[mode]
    ticks = np.arange(bounds[0], bounds[1]+0.01,0.05)
    ticklabels = ['{:.2f}'.format(tt) for tt in ticks]
    if bounds[0]>0: ticklabels[0] = '< {:.2f}'.format(bounds[0])
    ticklabels[-1] = '> {:.2f}'.format((bounds[1]))
    
    pc90 = df_city[f'km_share_{mode}'].quantile(0.9)
    df_city['marker']=(df_city['ln_population']-df_city.ln_population.min()+.01)*0.5

    fig, ax= plt.subplots(figsize=(10,6))
    
    # background for nodata countries
    df_country.plot(color='0.95', ax=ax)
    df_city.plot(markersize=0.015, color='0.7', ax=ax)
    
    # for the colorbar
    # https://geopandas.org/en/latest/docs/user_guide/mapping.html
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0)
    label='Walk share of km traveled' if mode=='on_foot' else 'Cycling share of km traveled'
    df_city.loc[df_city[f'km_share_{mode}']>=pc90].plot(f'km_share_{mode}', markersize='marker', 
                legend_kwds={"label": label}, cmap='Spectral', cax=cax, ax=ax, legend=True,vmin=bounds[0], vmax=bounds[1])
    cax.set_yticks(ticks, ticklabels)
    cax.tick_params(labelsize=12)
    cax.set_ylabel(label,fontsize=12)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')
    fig.tight_layout()
    fig.savefig(f'{figpath}/{mode}_top10pc_cities.png', dpi=1200)
    fig.savefig(f'{figpath}/{mode}_top10pc_cities.pdf', dpi=1200)

In [ ]:
for mode in ['cycling', 'on_foot']:
    fig, ax= plt.subplots(figsize=(10,6))
    # background for nodata countries
    df_country.plot(color='0.85', ax=ax)

    bounds = {'cycling':[0.0,0.03], 'on_foot':[0.0,0.05]}[mode]
    ticks = np.arange(bounds[0], bounds[1]+0.01,0.01)
    ticklabels = ['{:.2f}'.format(tt) for tt in ticks]
    if bounds[0]>0: ticklabels[0] = '< {:.2f}'.format(bounds[0])
    ticklabels[-1] = '> {:.2f}'.format((bounds[1]))
    
    # for the colorbar
    # https://geopandas.org/en/latest/docs/user_guide/mapping.html
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0)
    label='Walk share of km traveled' if mode=='on_foot' else 'Cycling share of km traveled'
    vmax = 0.03 if mode=='cycling' else 0.05
    df_country.plot(f'km_share_{mode}', cmap='Spectral', legend=True, legend_kwds={"label": label},
                    cax=cax, ax=ax, vmin=bounds[0], vmax=bounds[1])
    cax.tick_params(labelsize=12)
    cax.set_yticks(ticks, ticklabels)
    cax.set_ylabel(label,fontsize=12)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')
    fig.tight_layout()
    fig.savefig(f'{figpath}/{mode}_world.png')

In [ ]:
# boxplots of top 15 countries, with at least 10 cities
n_cities = df_city.groupby('country').size()
countries_to_use = n_cities[n_cities>=10].index.values

bike_countries = df_country.loc[countries_to_use].sort_values(by=y_bike, ascending=False)[[y_bike]].iloc[:15]
walk_countries = df_country.loc[countries_to_use].sort_values(by=y_walk, ascending=False)[[y_walk]].iloc[:15]

fig, axes= plt.subplots(1,2, figsize=(12,6))

df_city.sort_values(by='km_share_cycling_national', ascending=False, inplace=True)
sns.boxplot(df_city[(df_city.country.isin(bike_countries.index)) & (df_city.population>50000)], 
            y='country',x=y_bike, ax=axes[0], hue='country', palette='Spectral_r')
axes[0].scatter(y=range(15), x=bike_countries[y_bike], marker='o', color='0.3', zorder=2) # add country mean

axes[0].set_xticks(np.arange(0,0.3,0.1))
axes[0].set_xlabel('Cycling share of km traveled', fontsize=14)
axes[0].set_ylabel('')

df_city.sort_values(by='km_share_on_foot_national', ascending=False, inplace=True)
sns.boxplot(df_city[(df_city.country.isin(walk_countries.index)) & (df_city.population>50000)], 
            y='country',x=y_walk, ax=axes[1], hue='country', palette='Spectral_r')
axes[1].scatter(y=range(15), x=walk_countries[y_walk], marker='o', color='0.3', zorder=2)
axes[1].set_xticks(np.arange(0,0.4,0.1))
axes[1].set_xlabel('Walking share of km traveled', fontsize=14)
axes[1].set_ylabel('')
fig.tight_layout()
fig.savefig(f'{figpath}/boxplots_of_travelshare.png', dpi=1200)
fig.savefig(f'{figpath}/boxplots_of_travelshare.pdf')

#### Part 2: Model Results

In [ ]:
model_type='beta'
assert model_type in ['beta','preferred'] # variable list not valid for the others
boxplotDict={'intercept':'beta1','sndi':'beta2','density':'beta3','precipitation':'beta4','min_temp':'beta5',
             'bikelanes':'beta6','slope':'beta7','includes_inboundoutbound':'beta8','rail_in_city':'beta9','max_temp':'beta10','population':'beta11',
             'min_temp2':'beta12'}

coefficient_dict={'beta2':'sndi','beta3':'density','beta4':'precipitation','beta5':'min_temp',
                 'beta6':'bikelanes','beta7':'slope','beta8':'includes_inboundoutbound','beta9':'rail_in_city','beta10':'max_temp',
                  'beta11':'population','beta12':'min_temp2'}


sns.set_palette("Set2")
color_list=sns.color_palette("Set2").as_hex()
colorDict={} #i:color_list[abs(i-7)] for i in range(0,8)}
colorDict['bikelanes'] = '#a6d854' #1
colorDict['density'] = '#e78ac3' #2
colorDict['sndi'] = '#8da0cb' #3
colorDict['precipitation'] = '#fc8d62' #4
colorDict['rail_in_city'] = '#66c2a5' #5
colorDict['slope'] = '#b3b3b3' #6
colorDict['max_temp'] = '#e5c494' #7
colorDict['population'] = '#ffd92f' #8
colorDict['min_temp2'] = '#a6d854'
colorDict['motorways'] = '#e78ac3'
colorDict['sndi_added'] = '#8da0cb'
colorDict['dependency_ratio'] = '#fc8d62'
colorDict['gdp_per_capita'] = '#66c2a5'
colorDict['gas_prices'] = '#e5c494'

if model_type=='beta':
    recode_dict = {'dependency_ratio':'Dependency Ratio', 'sndi':'Street Connectivity', 'precipitation':'Precipitation', 'rail_in_city':'Rail Service',
        'bikelanes':'Bicycle Lanes', 'density':'Population Density', 'slope':'Steep Terrain', 'gdp_per_capita':'GDP per Capita', 'gas_prices':'Gasoline Prices','max_temp':'Maximum Temp',
              'sndi_added':'Added Street Connectivity','population':'Population','motorways':'Motorways','min_temp2':'Min Temp (nonlinear)'}    
else:
    recode_dict = {'min_temp':'Minimum Temp', 'dependency_ratio':'Dependency Ratio', 'sndi':'Street Connectivity', 'precipitation':'Precipitation', 'rail_in_city':'Rail Service',
        'bikelanes':'Bicycle Lanes', 'density':'Population Density', 'slope':'Steep Terrain', 'gdp_per_capita':'GDP per Capita', 'gas_prices':'Gasoline Prices','max_temp':'Maximum Temp',
              'sndi_added':'Added Street Connectivity','population':'Population','motorways':'Motorways','min_temp2':'Min Temp Squared'}    

# for beta, the squared term is combined effect, so drop the linear term
cols_to_drop = ['includes_inboundoutbound', 'min_temp'] if model_type=='beta' else ['includes_inboundoutbound']
betas_to_plot=[key for key, value in coefficient_dict.items() if value not in cols_to_drop]

In [ ]:
# get median estimates for every estimate of city-level data
city_level_variables=pd.read_csv(f'{outputpath}/beta_estimates_{model_type}.csv')
city_level_variables=city_level_variables[[col for col in city_level_variables.columns if "50%" in col]].copy()
city_level_variables.columns=[col[:-4] for col in city_level_variables.columns]

dfs = {}

# get median estimates for every country-level data on intercept 
country_level_estimates=pd.read_csv(f'{outputpath}/gamma_estimates_{model_type}.csv')
country_level_estimates=country_level_estimates[['country_var']+[col for col in country_level_estimates.columns if "50%" in col]].copy()

# get data for each individual density plot
for beta in betas_to_plot:
    city_level_variables[f'{beta}_walk_rounded']=round(city_level_variables[f'{beta}_walk']*20,1)/20
    dfs[f'{beta}_walk_rounded']=city_level_variables.groupby(f'{beta}_walk_rounded').size()
    dfs[f'{beta}_walk_rounded'].name=coefficient_dict[f'{beta}']

# join into a single dataframe to plot
walk_coefficients=dfs['beta2_walk_rounded'].to_frame()
for beta in [beta for beta in betas_to_plot if beta != 'beta2']:
    walk_coefficients=walk_coefficients.join(dfs[f'{beta}_walk_rounded'],how='outer')
    
# determine order
walk_order=city_level_variables[[col+'_walk' for col in betas_to_plot]]
walk_order=walk_order.median(numeric_only=True).to_frame().reset_index()
walk_order['index']=walk_order['index'].str.replace('_walk','')
walk_order['index']=walk_order['index'].replace(coefficient_dict)
walk_order.columns=['variable','abs_median']
walk_order['abs_median']=abs(walk_order['abs_median'])
walk_order.sort_values('abs_median',inplace=True)

# now add in the estimates for country-level data
walk_country_level = country_level_estimates[(country_level_estimates.country_var.str.endswith('walk')) & (country_level_estimates.country_var!='intercept_walk')] 
walk_country_level['country_var'] = walk_country_level['country_var'].str.replace('_walk','')
walk_country_level.columns=['variable','abs_median']
walk_order_final=pd.concat([walk_order,walk_country_level]).sort_values('abs_median')
walk_coefficients['gdp_per_capita']=0
walk_coefficients['dependency_ratio']=0
walk_coefficients['gas_prices']=0

# order df to plot
walk_coefficients=walk_coefficients[walk_order_final.variable.to_list()]

# do the plot
plt.figure(figsize=(6, 6))
plt.axvline(x=0, color='#63666A', linestyle='--')
for i, (variable, city_level_variables) in enumerate(walk_coefficients.items()):
    if variable not in ['gdp_per_capita','gasoline_prices','dependency_ratio']:
        nonnull=(walk_coefficients[walk_coefficients[variable]>0].index.tolist())
        plt.scatter(nonnull,np.full(len(nonnull),i*100), color=colorDict[variable], marker='_')
for i, (variable, city_level_variables) in enumerate(walk_coefficients.items()):
    if variable not in ['gdp_per_capita','gas_prices','dependency_ratio']:
        plt.fill_between(walk_coefficients.index, walk_coefficients[variable]+100*i,y2=100*i,color=colorDict[variable])
for i, (variable, city_level_variables) in enumerate(walk_coefficients.items()):
    if variable in ['gdp_per_capita','gas_prices','dependency_ratio']:
        val_to_plot=walk_order_final[walk_order_final['variable']==variable].iloc[0].abs_median
        plt.scatter(val_to_plot,i*100,marker='s',s=50,color=colorDict[variable])    
        
plt.xlabel('Effect of 1 standard deviation change on walk share')
plt.yticks([100 * i for i in range(len(walk_coefficients.columns))], [recode_dict[item] for item in walk_coefficients.columns.to_list()])
plt.text(0.05, 0.94,'B. Walking', size=14, ha='left', transform=plt.gca().transAxes)

plt.tight_layout()
plt.savefig(f'{figpath}/{model_type}_coefficient_distribution_walking.png',bbox_inches='tight', dpi=1200)
plt.savefig(f'{figpath}/{model_type}_coefficient_distribution_walking.pdf',bbox_inches='tight')

In [ ]:
# get median estimates for every estimate of city-level data
city_level_variables=pd.read_csv(f'{outputpath}/beta_estimates_{model_type}.csv')
city_level_variables=city_level_variables[[col for col in city_level_variables.columns if "50%" in col]].copy()
city_level_variables.columns=[col[:-4] for col in city_level_variables.columns]

# get data for each individual density plot
for beta in betas_to_plot:
    city_level_variables[f'{beta}_bike_rounded']=round(city_level_variables[f'{beta}_bike']*50,1)/50
    dfs[f'{beta}_bike_rounded']=city_level_variables.groupby(f'{beta}_bike_rounded').size()
    dfs[f'{beta}_bike_rounded'].name=coefficient_dict[f'{beta}']

# join into a single dataframe to plot
bike_coefficients=dfs['beta2_bike_rounded'].to_frame()
for beta in [beta for beta in betas_to_plot if beta != 'beta2']:
    bike_coefficients=bike_coefficients.join(dfs[f'{beta}_bike_rounded'],how='outer')
    
# determine order
bike_order=city_level_variables[[col+'_bike' for col in betas_to_plot]]
bike_order=bike_order.median(numeric_only=True).to_frame().reset_index()
bike_order['index']=bike_order['index'].str.replace('_bike','')
bike_order['index']=bike_order['index'].replace(coefficient_dict)
bike_order.columns=['variable','abs_median']
bike_order['abs_median']=abs(bike_order['abs_median'])
bike_order.sort_values('abs_median',inplace=True)

# now add in the estimaets for country-level data
bike_country_level = country_level_estimates[(country_level_estimates.country_var.str.endswith('bike')) & (country_level_estimates.country_var!='intercept_bike')] 
bike_country_level['country_var'] = bike_country_level['country_var'].str.replace('_bike','')

bike_country_level.columns=['variable','abs_median']
bike_order_final=pd.concat([bike_order,bike_country_level]).sort_values('abs_median')
bike_coefficients['gdp_per_capita']=0
bike_coefficients['dependency_ratio']=0
bike_coefficients['gas_prices']=0

# order df to plot
bike_coefficients=bike_coefficients[bike_order_final.variable.to_list()]

# do the plot
plt.figure(figsize=(6, 6))
plt.axvline(x=0, color='#63666A', linestyle='--')
for i, (variable, city_level_variables) in enumerate(bike_coefficients.items()):
    if variable not in ['gdp_per_capita','gasoline_prices','dependency_ratio']:
        nonnull=(bike_coefficients[bike_coefficients[variable]>0].index.tolist())
        plt.scatter(nonnull,np.full(len(nonnull),i*100), color=colorDict[variable], marker='_')
for i, (variable, city_level_variables) in enumerate(bike_coefficients.items()):
    if variable not in ['gdp_per_capita','gas_prices','dependency_ratio']:
        plt.fill_between(bike_coefficients.index, bike_coefficients[variable]+100*i,y2=100*i,color=colorDict[variable])
for i, (variable, city_level_variables) in enumerate(bike_coefficients.items()):
    if variable in ['gdp_per_capita','gas_prices','dependency_ratio']:
        val_to_plot=bike_order_final[bike_order_final['variable']==variable].iloc[0].abs_median
        plt.scatter(val_to_plot,i*100,marker='s',s=50,color=colorDict[variable])    
        
plt.xlabel('Effect of 1 standard deviation change on cycling share')
plt.yticks([100 * i for i in range(len(bike_coefficients.columns))], [recode_dict[item] for item in bike_coefficients.columns.to_list()])
plt.text(0.96, 0.94,'A. Cycling', ha='right', size=15, transform=plt.gca().transAxes)
plt.tight_layout()
plt.savefig(f'{outputpath}/{model_type}_coefficient_distribution_biking.png',bbox_inches='tight', dpi=1200)
plt.savefig(f'{outputpath}/{model_type}_coefficient_distribution_biking.pdf',bbox_inches='tight')


In [ ]:
# get median estimates for every estimate of city-level data
city_level_variables=pd.read_csv(f'{outputpath}/beta_estimates_{model_type}.csv')
city_level_variables=city_level_variables[[col for col in city_level_variables.columns if "50%" in col]].copy()
city_level_variables.columns=[col[:-4] for col in city_level_variables.columns]

# make df of bike data relabeled
bike_col=[col for col in city_level_variables.columns if '_bike' in col]
bike_estimates=city_level_variables[bike_col].copy()
bike_estimates.columns=[col[:-5] for col in bike_estimates.columns]
bike_estimates.columns = [coefficient_dict.get(col, col) for col in bike_estimates.columns]

# make df of walk data relabeled
walk_col=[col for col in city_level_variables.columns if '_walk' in col]
walk_estimates=city_level_variables[walk_col].copy()
walk_estimates.columns=[col[:-5] for col in walk_estimates.columns]
walk_estimates.columns = [coefficient_dict.get(col, col) for col in walk_estimates.columns]

# add in countries
walk_estimates=pd.read_csv('data/country_23.csv')[['country']].join(walk_estimates)
bike_estimates=pd.read_csv('data/country_23.csv')[['country']].join(bike_estimates)

In [ ]:
dfs['walk_df'] = countries.set_index('NAME_EN')[['geometry']].join(walk_estimates.set_index('country'), how='left')
dfs['bike_df'] = countries.set_index('NAME_EN')[['geometry']].join(bike_estimates.set_index('country'), how='left')

dfs['bike_df']=dfs['bike_df'].join(df_country.cycling_share_including_transit)
dfs['walk_df']=dfs['walk_df'].join(df_country.on_foot_share_including_transit)

def continuous_map(variable,mode):
    mname = {'bike':'cycling','walk':'walking'}[mode]
    vname = {'density':'density','bikelanes':'bicycle lanes'}[variable]
    
    fig, ax= plt.subplots(figsize=(10,6))
    label=f'Effect of {vname} on {mname} share'
    dfs[f'{mode}_df'].plot(ax=ax,color='0.85')
    if mode=='walk': #.075 for modeshare
        dfs[f'{mode}_df'].plot(ax=ax,column=variable,cmap='Spectral',
                vmin=-0.05,vmax=0.05,legend=True,legend_kwds={"label": label, 'ticks':np.arange(-0.04,0.05,0.02)})
    if mode=='bike':
        dfs[f'{mode}_df'].plot(ax=ax,column=variable,cmap='Spectral',
                vmin=-0.01,vmax=0.01,legend=True,legend_kwds={"label": label, 'ticks':np.arange(-0.01,0.011,0.005)})    
    ax.axis('off')
    ax.set_ylabel(label,fontsize=12)
    plot_title = 'A. Cycling' if mode=='bike' else 'B. Walking'
    ax.text(0.98, 1.05, plot_title, ha='right', size=15, transform=ax.transAxes)
    plt.savefig(f'{figpath}/{model_type}_country_coefficients_{variable}_{mode}.png',bbox_inches='tight');
    
for variable in ['density', 'bikelanes']:
    for mode in ['walk','bike']:
        continuous_map(variable,mode)
        #print(f'\n{variable}, {mode}')
        #print(dfs[f'{mode}_df'].sort_values(by=variable, ascending=False)[variable].head(10))


In [ ]:
gdp_on_density=df_city.groupby('country')['ln_gdp'].mean().to_frame()
gdp_on_density=gdp_on_density.join(pd.read_csv('data/country_23.csv')[['country','population']].join(city_level_variables).set_index('country')[['beta3_bike','beta3_walk','population']])

# get ticks in natural units
xticks = [500, 3000, 20000, 150000]

fig, ax1 = plt.subplots(figsize=(6, 5))
ax2 = ax1.twinx()
sns.regplot(data=gdp_on_density,x='ln_gdp',y='beta3_walk',ax=ax1,color='#8da0cb',scatter_kws={'s':10})
sns.regplot(data=gdp_on_density,x='ln_gdp',y='beta3_bike',ax=ax2,color='#66c2a5',scatter_kws={'s':10})
ax1.set_ylabel('Impact of Density on Walk Share (Purple)', fontsize=12)  # Adding the x-axis label
ax2.set_ylabel('Impact of Density on Bicycle Share (Teal)', fontsize=12)
ax1.set_xlabel('GDP per Capita (log scale)', fontsize=12)  # Adding the x-axis label
ax1.set_xticks([np.log(xx) for xx in xticks], [str(xx) for xx in xticks])
ax1.text(12.2,0.028,'Walk',color='#8da0cb', weight='bold',size=12, ha='right')
ax1.text(12.2,0.00,'Bicycle',color='#66c2a5', weight='bold',size=12, ha='right')

ax1.set_ylim(top=0.06)
ax2.set_ylim(top=0.025)

plt.savefig(f'{figpath}/{model_type}_gdp_on_density_coefficient.png',bbox_inches='tight')
plt.savefig(f'{figpath}/{model_type}_gdp_on_density_coefficient.pdf',bbox_inches='tight')



In [ ]:
city_level_variables

In [ ]:
# new way, for a beta model, just plot the country-level marginal effects
# get  estimates for every estimate of city-level data
# these are called beta, but they are marginal effects

vname, betaname = 'density', 'beta3'

city_level_variables=pd.read_csv(f'{outputpath}/beta_estimates_{model_type}.csv')
city_level_variables=city_level_variables[[col for col in city_level_variables.columns if "50%" in col]].copy()
city_level_variables.columns=[col[:-4] for col in city_level_variables.columns]

estimates=pd.read_csv('data/country_23.csv')[['country']].join(city_level_variables)
estimates.set_index('country',inplace=True)
estimates.columns=estimates.columns.str.replace('bike','cycling').str.replace('walk','on_foot')

countries = ['Germany','Japan','United States','Colombia','India','Nigeria',]
cnames = [cc if cc!='United States' else 'USA' for cc in countries ]

estimates.rename(columns={betaname+'_on_foot':'Walking',betaname+'_cycling':'Cycling'}, inplace=True) # for legend
fig, ax = plt.subplots(figsize=(6, 5))
estimates.loc[countries][['Walking','Cycling']].plot(kind='bar', color=['#66c2a5','#fc8d62'], ax=ax)
ax.set_xticklabels(cnames, rotation=0)
ax.set_xlabel('')
ax.set_yticks(np.arange(0,0.041, 0.01))
ax.set_ylabel('Effect of 1 standard deviation increase in '+vname)
#labels=['Walking','Cycling'])
ax.legend(fontsize=12)

fig.tight_layout()
fig.savefig(f'{figpath}/{model_type}_{vname}_me.png', dpi=600)


In [ ]:
# coefficient comparisons
coeffs = {}
df = pd.DataFrame()
models = ['trips','high_gdp','inoutbound_only','nonlinear','saturated','linear','beta',]
namedict = {'trips':'trip share', 'high_gdp': 'high GDP', 'inoutbound_only':'with in- and outbound', 'beta':'preferred'}
modelnames = [namedict[mm] if mm in namedict else mm for mm in models ]

colors = ['#873e23','#e78ac3', '#a6d854', '#ffd92f', '#fc8d62', '#8da0cb','k']
markers = ['o','^','v','s','d','*','x']
# load in coefficient tables
for model in models:
    coeffs[model] = pd.read_csv(outputpath+'Coefficient_table_{}.csv'.format(model), index_col=0)
    coeffs[model].index.name = 'parameter'
    for mode in ['walk','bike']: # put text in columns as floats
        coeffs[model][model+'_'+mode+'_median'] = coeffs[model][mode].apply(lambda x: float(x.split()[0]))
        coeffs[model][model+'_'+mode+'_05'] = coeffs[model][mode].apply(lambda x: float(x.split()[1][1:-1]))
        coeffs[model][model+'_'+mode+'_95'] = coeffs[model][mode].apply(lambda x: float(x.split()[2][:-1]))
    coeffs[model].drop(columns=['walk','bike'], inplace=True)
    df = df.join(coeffs[model], how='outer')
df.drop(['beta intercept','gamma intercept',], inplace=True) # intercepts don't have the same meaning for beta model
df['n'] = range(len(df),0,-1) # integer for plot
df.reset_index(inplace=True)

# better names for coefficients
vnames = {'beta intercept': 'City intercept', 'bikelanes': 'Bicycle facilities', 'density': 'Pop. density', 'density2': 'Pop. density (quadratic)', 
          'gamma intercept': 'Country intercept', 'gas_price': 'Gasoline price', 'gdp_per_capita': 'GDP per capita', 
          'includes_inboundoutbound': 'Includes in- and out-bound trips', 'max_temp': 'Maximum temperature', 'max_temp2': 'Max. temperature (quadratic)', 
          'min_temp': 'Minimum temperature', 'min_temp2': 'Min. temperature (quadratic)', 'rail in_city': 'rail', 'sndi': 'Street connectivity',
          'sndi_added': 'Street connectivity added'}
df.parameter = df.parameter.apply(lambda x: vnames[x] if x in vnames else x.title().replace('_',' '))

fig,axes = plt.subplots(1,2, figsize=(7,7))
space=len(models)+3

# add all of the markers at the average estimate
for i, model in enumerate(models):
    axes[0].scatter(x=df[model+'_bike_median'], y=(df.n)*space+i, color=colors[i], marker=markers[i], s=5)
    axes[1].scatter(x=df[model+'_walk_median'], y=(df.n)*space+i, color=colors[i], marker=markers[i], s=5)

    for index, row in df.iterrows():
        axes[0].plot([row[model+'_bike_05'], row[model+'_bike_95']], [row.n*space+i, row.n*space+i], color=colors[i])
        axes[1].plot([row[model+'_walk_05'], row[model+'_walk_95']], [row.n*space+i, row.n*space+i], color=colors[i])

for ax, xlabel in zip(axes, ['Bicycle','Walk']):
    ax.set_xlabel(xlabel)
    # add vertical line at 0
    ax.axvline(0, color='grey')
    ax.set_yticks(df.n * space +2)
axes[0].set_yticklabels(df.parameter, size=9)
axes[1].set_yticklabels([])

# add legend
legend_elements = [(Line2D([0],[0],marker=markers[i], color=colors[i], linestyle='-', label=modelnames[i], markersize=5)) for i in range(len(models)-1,-1,-1)]


fig.legend(handles=legend_elements,loc='center left', bbox_to_anchor=(1,0.5));
fig.tight_layout()
fig.savefig(f'{figpath}/coefficient_comparisons.pdf',bbox_inches='tight')